In [14]:
# Data Flattening
import numpy as np
import pandas as pd

import datetime
from datetime import datetime

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, ArrayType, FloatType, MapType, LongType
from pyspark.sql.functions import lit, udf, size, avg, min as min_, max as max_, sum as sum_, count, countDistinct, col, mean, stddev, struct, explode, explode_outer, unix_timestamp, sum as sum_
from operator import itemgetter
import json

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

from pyspark.ml.feature import Bucketizer

from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StandardScaler

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, row_number, dense_rank, lead, lag, rank
from pyspark.sql.window import Window

from pyspark.ml.classification import LogisticRegressionModel

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

import collections
from collections import deque
import redis
import json


In [15]:
!pip install numpy pandas sklearn redis

In [16]:
'''
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt=2019-09-01/hr=01/*')
rdd = df.rdd
r = rdd.map(lambda x: [(x['id'], x['when'], t['name'], t['rx_bytes'],t['tx_bytes'],t['rx_bps'],t['tx_bps'],t['rx_packets'],t['tx_packets']) for t in x['device']['interfaces']])
r1 = r.flatMap(lambda x: [t for t in x])
df1 = sqlContext.createDataFrame(r1)
df1 = df1.withColumnRenamed('_1','id')
tsudf = udf(lambda x: x[0]*1000000000+x[1], LongType())
df1 = df1.withColumn('ts', tsudf(struct(df1['_2']['seconds'],df1['_2']['nanos'])))
df1 = df1.withColumnRenamed('_3','name') \
        .withColumnRenamed('_4','rx_bytes') \
        .withColumnRenamed('_5','tx_bytes') \
        .withColumnRenamed('_6','rx_bps') \
        .withColumnRenamed('_7','tx_bps') \
        .withColumnRenamed('_8','rx_pkts') \
        .withColumnRenamed('_9','tx_pkts') \
        .drop('_2')

id_name_gp = df1.groupby('id','name').agg(sum_('rx_bytes'),sum_('tx_bytes'),sum_('rx_bps'),sum_('tx_bps'),sum_('rx_pkts'),sum_('tx_pkts'))
id_gp = df1.groupby('id').agg(sum_('rx_bytes'),sum_('tx_bytes'),sum_('rx_bps'),sum_('tx_bps'),sum_('rx_pkts'),sum_('tx_pkts'))
'''

"\ndf = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt=2019-09-01/hr=01/*')\nrdd = df.rdd\nr = rdd.map(lambda x: [(x['id'], x['when'], t['name'], t['rx_bytes'],t['tx_bytes'],t['rx_bps'],t['tx_bps'],t['rx_packets'],t['tx_packets']) for t in x['device']['interfaces']])\nr1 = r.flatMap(lambda x: [t for t in x])\ndf1 = sqlContext.createDataFrame(r1)\ndf1 = df1.withColumnRenamed('_1','id')\ntsudf = udf(lambda x: x[0]*1000000000+x[1], LongType())\ndf1 = df1.withColumn('ts', tsudf(struct(df1['_2']['seconds'],df1['_2']['nanos'])))\ndf1 = df1.withColumnRenamed('_3','name')         .withColumnRenamed('_4','rx_bytes')         .withColumnRenamed('_5','tx_bytes')         .withColumnRenamed('_6','rx_bps')         .withColumnRenamed('_7','tx_bps')         .withColumnRenamed('_8','rx_pkts')         .withColumnRenamed('_9','tx_pkts')         .drop('_2')\n\nid_name_gp = df1.groupby('id','name').agg(sum_('rx_bytes'),sum_('tx_bytes'),sum_('rx_bps'),sum_('tx_bps'),sum_('rx_pkts

In [17]:
#flatten interface
date = '2019-09-18'
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt='+date+'/*')

import uuid
def bbfunc(x):
    if len(x) == 0: return ''
    else: return str(uuid.UUID(bytes=bytes(x)))
bb_udf = udf(bbfunc, StringType())

def macfunc(bb):
    x = ''
    for i in range(0,6):
        x += '%0*X' % (2, bytes(bb)[i]) + '-'
    return x[:-1]
mac_udf = udf(macfunc, StringType())

df = df.withColumn('id', mac_udf('id')) \
        .withColumn('mac', mac_udf('mac')) \
        .withColumn('org_id', bb_udf('org_id')) \
        .withColumn('site_id', bb_udf('site_id'))
                    
ts_udf = udf(lambda x: x[0]*1000000+int(x[1]/1000), LongType())
df = df.withColumn('ts', ts_udf(struct('when.seconds', 'when.nanos'))).drop('when')

dfin = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 explode_outer('device.interfaces').alias('interfaces'))

dfin = dfin.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                  'hostname',
                  'model',
                  'version',
                  'serial',
                  'uptime',
                   'pctrl_ix',
                   'pctrl_maxpwr',
                   'pctrl_consump',
                   'pctrl_gband',
                  col('interfaces.name').alias('name'),
                   col('interfaces.link').alias('link'),
                   col('interfaces.full_duplex').alias('full_duplex'),
                   col('interfaces.mbps').alias('mbps'),
                   col('interfaces.mtu').alias('mtu'),
                   col('interfaces.address').alias('address'),
                   col('interfaces.admin_status').alias('admin_status'),
                   col('interfaces.last_flapped.seconds').alias('last_flapped'),
                   col('interfaces.errors').alias('errors'),
                   col('interfaces.poe.enabled').alias('poe_enabled'),
                   col('interfaces.poe.status').alias('poe_status'),
                   col('interfaces.poe.power_limit').alias('poe_pwrlimit'),
                   col('interfaces.poe.power').alias('poe_power'),
                   col('interfaces.poe.priority').alias('poe_priority'),
                   col('interfaces.poe.class').alias('poe_class'),
                   col('interfaces.poe.mode').alias('poe_mode'),
                   col('interfaces.rx_bytes').alias('rx_bytes'),
                   col('interfaces.tx_bytes').alias('tx_bytes'),
                   col('interfaces.rx_packets').alias('rx_pkts'),
                   col('interfaces.tx_packets').alias('tx_pkts'),
                   col('interfaces.rx_bps').alias('rx_bps'),
                   col('interfaces.tx_bps').alias('tx_bps'))

dfin.persist().first()
dfin.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/interfaces'+date)
dfin.unpersist()

Py4JJavaError: An error occurred while calling o173.parquet.
: java.nio.file.AccessDeniedException: s3://mist-data-science-dev/jing/switch/interfaces2019-09-18/_temporary/0: innerMkdirs on s3://mist-data-science-dev/jing/switch/interfaces2019-09-18/_temporary/0: com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: AC3819AC759D2C1B), S3 Extended Request ID: aTpmWWZFqEldgOzeSUGlthJlZwW3LIhLeHqMvEee0HsNBx5OOhS445ZDhZLayj5sSzialklXPBo=
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:158)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:101)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.mkdirs(S3AFileSystem.java:1484)
	at org.apache.hadoop.fs.FileSystem.mkdirs(FileSystem.java:1914)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.setupJob(FileOutputCommitter.java:343)
	at org.apache.spark.internal.io.HadoopMapReduceCommitProtocol.setupJob(HadoopMap

In [ ]:
#flatten chassis: flatten all 4 arrays
dfch = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.chassis.routing_engines').alias('routing_engines1'),
                 col('device.chassis.fans').alias('fans1'),
                 col('device.chassis.temperatures').alias('temperatures1'),
                 col('device.chassis.psus').alias('psus1')
                )

dfch = dfch.select(col('*'),explode_outer('routing_engines1').alias('routing_engines')) \
            .select(col('*'),explode_outer('temperatures1').alias('temperatures')) \
            .select(col('*'),explode_outer('fans1').alias('fans')) \
            .select(col('*'),explode_outer('psus1').alias('psus')) \
            .drop('routing_engines1','temperatures1','fans1','psus1')

dfch = dfch.select(col('*'),
                  col('routing_engines.slot').alias('eng_slot'),
                  col('routing_engines.is_master').alias('eng_ismaster'),
                  col('routing_engines.status').alias('eng_status'),
                  col('routing_engines.temp_celsius').alias('eng_tempc'),
                  col('routing_engines.memory.total_mb').alias('eng_memoryttlmb'),
                  col('routing_engines.memory.in_use_mb').alias('eng_memoryinusemb'),
                  col('routing_engines.cpu.user').alias('eng_cpuuser'),
                  col('routing_engines.cpu.background').alias('eng_cpubackground'),
                  col('routing_engines.cpu.system').alias('eng_cpusystem'),
                  col('routing_engines.cpu.interrupt').alias('eng_cpuinterrupt'),
                  col('routing_engines.cpu.idle').alias('eng_cpuidle'),
                  col('routing_engines.cputemp_celsius').alias('eng_cputempc'),
                  col('routing_engines.model').alias('eng_model'),
                  col('routing_engines.last_reboot_reason').alias('eng_lastrebootreason'),
                  col('routing_engines.load_avg').alias('eng_loadavg'),
                  col('fans.name').alias('fans_name'),
                  col('fans.status').alias('fans_status'),
                   col('temperatures.name').alias('temp_name'),
                   col('temperatures.status').alias('temp_status'),
                   col('temperatures.temp_celsius').alias('temp_tempc'),
                    col('psus.name').alias('psus_name'),
                   col('psus.status').alias('psus_status')) \
                    .drop('routing_engines','fans','temperatures','psus')

dfch.persist().first()
dfch.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/chassis'+date)
dfch.unpersist()

In [ ]:
#flatten config_history
dfconf = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.config_history').alias('config_history1')
                )

dfconf = dfconf.select(col('*'),explode_outer('config_history1').alias('config_history')) \
            .drop('config_history1')

dfconf = dfconf.select(col('*'),
                  col('config_history.sequence_num').alias('confhist_seqnum'),
                  col('config_history.user').alias('confhist_user'),
                   col('config_history.client').alias('confhist_client'),
                   col('config_history.diff').alias('confhist_diff'),
                   col('config_history.timestamp').alias('confhist_timestamp')) \
                    .drop('config_history')

dfconf.persist().first()
dfconf.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/config_history'+date)
dfconf.unpersist()

In [ ]:
#flatten alarms
dfalarm = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.alarms').alias('alarms1')
                )

dfalarm = dfalarm.select(col('*'),explode_outer('alarms1').alias('alarms')) \
            .drop('alarms1')

dfalarm = dfalarm.select(col('*'),
                  col('alarms.timestamp').alias('alarm_ts'),
                  col('alarms.type').alias('alarm_type'),
                   col('alarms.alarm_class').alias('alarm_class'),
                   col('alarms.description').alias('alarm_desc'),
                   col('alarms.short_description').alias('alarm_shortdesc')) \
                    .drop('alarms')

dfalarm.persist().first()
dfalarm.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/alarms'+date)
dfalarm.unpersist()

In [ ]:
#flatten ipv4_route
dfipv = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.ipv4_route').alias('ipv1')
                )

dfipv = dfipv.select(col('*'),explode_outer('ipv1').alias('ipv')) \
            .drop('ipv1')

dfipv = dfipv.select(col('*'),
                  col('ipv.dst.mask').alias('ipv_dstmask'),
                  col('ipv.dst.ip').alias('ipv_dstip'),
                   col('ipv.gw').alias('ipv_gw')) \
                    .drop('ipv')

dfipv.persist().first()
dfipv.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/ipv'+date)
dfipv.unpersist()

In [ ]:
#flatten lldpneighbors
dflldp = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.lldpneighbors').alias('lldp1')
                )

dflldp = dflldp.select(col('*'),explode_outer('lldp1').alias('lldp')) \
            .drop('lldp1')

dflldp = dflldp.select(col('*'),
                  col('lldp.local_port_id').alias('lldp_locportid'),
                  col('lldp.local_parent_iface_name').alias('lldp_locifacename'),
                   col('lldp.remote_chassis_id').alias('lldp_rmtchassisid'),
                   col('lldp.remote_chassis_idsubtype').alias('lldp_rmtchassisidsub'),
                      col('lldp.remote_port_desc').alias('lldp_rmtportdesc'),
                      col('lldp.remote_system_name').alias('lldp_rmtsysname'),
                      col('lldp.remote_mgmt_addr').alias('lldp_rmtmgmtaddr'),
                      col('lldp.remote_system_desc').alias('lldp_rmtsysdesc')) \
                        .drop('lldp')

dflldp.persist().first()
dflldp.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/lldp'+date)
dflldp.unpersist()

In [ ]:
#flatten svistats
dfsvi = df.select('id',
                 'mac',
                 'org_id',
                 'site_id',
                 'ts',
                 'remote_addr',
                 col('device.hostname').alias('hostname'),
                 col('device.model').alias('model'),
                 col('device.firmware_version').alias('version'),
                 col('device.serial_number').alias('serial'),
                 col('device.uptime').alias('uptime'),
                 col('device.poe_controller.index').alias('pctrl_ix'),
                 col('device.poe_controller.max_power').alias('pctrl_maxpwr'),
                 col('device.poe_controller.consumption').alias('pctrl_consump'),
                 col('device.poe_controller.guardband').alias('pctrl_gband'),
                 col('device.svistats').alias('svi1')
                )

dfsvi = dfsvi.select(col('*'),explode_outer('svi1').alias('svi')) \
            .drop('svi1')

dfsvi = dfsvi.select(col('*'),
                  col('svi.dev').alias('svi_dev'),
                  col('svi.vlan').alias('svi_vlan'),
                   col('svi.ips').alias('svi_ips')) \
                    .drop('svi')

dfsvi.persist().first()
dfsvi.coalesce(100).write.parquet('s3://mist-data-science-dev/jing/switch/svi'+date)
dfsvi.unpersist()

In [ ]:
# data missing check
dfch.groupby('id').agg(countDistinct('ts')).show() 
w0 = Window.partitionBy('id').orderBy('ts')                                                               
df0 = df.select('id','ts').distinct()                                                                     
df0 = df0.withColumn('ts1', lag('ts').over(w0))                                                           
df0 = df0.na.fill(0)                                                                                      
df0 = df0.withColumn('delta', df0.ts - df0.ts1)                                                            
df0.filter(df0.delta>70000000).sort('delta').show(100)  

In [ ]:
"""
{'name': 'ge-0/0/0',
 'link': False,
 'full_duplex': False,
 'mbps': 1000,
 'mtu': 1514,
 'address': bytearray(b'<\x8c\x93\x94\x95\xaf'),
 'admin_status': True,
 'last_flapped': Row(seconds=260, nanos=0),
 'errors': [],
 'poe': Row(enabled=True, status=False, power_limit=15.399999618530273, power=0.0, priority=False, class='not-applicable', mode=''),
 'rx_bytes': 0,
 'rx_bps': 0,
 'rx_packets': 0,
 'tx_bytes': 0,
 'tx_bps': 0,
 'tx_packets': 0}

"""
def macfunc(bb):
    x = ''
    for i in range(0,6):
        x += '%0*X' % (2, bytes(bb)[i]) + '-'
    return x[:-1]

r = rdd.map(lambda x: [(macfunc(x['id']), t['name'], x['when'], t['full_duplex'],t['last_flapped'])  for t in x['device']['interfaces'] if t['link']])




In [ ]:
df = spark.read.parquet('s3://mist-secorapp-staging/oc-stats/oc-stats-staging/dt=2019-09-23/*')
rdd = df.rdd

r = rdd.map(lambda x: [(x['id'], x['when'], t['name'], t['rx_bytes'],t['tx_bytes'],t['rx_bps'],t['tx_bps'],t['rx_packets'],t['tx_packets'],t['mbps'], t['mtu'], t['errors'], t['poe']) for t in x['device']['interfaces']])
r1 = r.flatMap(lambda x: [t for t in x])
#excluding errors for now
r2 = r1.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[12]['enabled'],x[12]['status'],x[12]['power_limit'],x[12]['power'],x[12]['priority'],x[12]['class'],x[12]['mode']))
            
df1 = sqlContext.createDataFrame(r2)
tsudf = udf(lambda x: x[0]*1000000000+x[1], LongType())
df1 = df1.withColumn('ts', tsudf(struct(df1['_2']['seconds'],df1['_2']['nanos']))).drop('_2')
df1 = df1.withColumnRenamed('_1','id') \
        .withColumnRenamed('_3','name') \
        .withColumnRenamed('_4','rx_bytes') \
        .withColumnRenamed('_5','tx_bytes') \
        .withColumnRenamed('_6','rx_bps') \
        .withColumnRenamed('_7','tx_bps') \
        .withColumnRenamed('_8','rx_pkts') \
        .withColumnRenamed('_9','tx_pkts') \
        .withColumnRenamed('_10','mbps') \
        .withColumnRenamed('_11','mtu') \
        .withColumnRenamed('_12','poe_enabled') \
        .withColumnRenamed('_13','poe_status') \
        .withColumnRenamed('_14','poe_power_limit') \
        .withColumnRenamed('_15','poe_power') \
        .withColumnRenamed('_16','poe_priority') \
        .withColumnRenamed('_17','poe_class') \
        .withColumnRenamed('_18','poe_mode')
#r1 = rdd.map(lambda x: [(x['id'], x['when'], t['poe_controller']['max_power'], t['']) for t in x['device']])


In [2]:
1. each interface has a configued mbps (id, name), not each id (800-1000), should be max bps
   446 unique interface (57/446 changed mbps in a day but in very small amount 10e-14 though)
   same for mtu (1500 or so)
2. pkts, bytes are incremental
3. bytearray(b'<\x8c\x93\x94\x95\xac') == [3C 8C 93 94 95 AC] ??? id
4. dont know mbps for id
5. mbps is mega bps configured for id (but under interfaces... should change to id level?)
6. 

[]

In [ ]:
w = Window.partitionBy('id','name').orderBy('ts')
df1 = df1.withColumn('prev_rxbytes', lag('rx_bytes').over(w)) \
        .withColumn('prev_txbytes', lag('tx_bytes').over(w)) \
        .withColumn('prev_rxpkts', lag('rx_pkts').over(w)) \
        .withColumn('prev_txpkts', lag('tx_pkts').over(w)) 
df1 = df1.filter(df1.prev_rxpkts.isNotNull())

df1 = df1.withColumn('rxpkts', df1.rx_pkts - df1.prev_rxpkts) \
            .withColumn('txpkts', df1.tx_pkts - df1.prev_txpkts) \
            .withColumn('rxbytes', df1.rx_bytes - df1.prev_rxbytes) \
            .withColumn('txbytes', df1.tx_bytes - df1.prev_txbytes) \
            .drop('prev_rxpkts','prev_txpkts','prev_rxbytes','prev_txbytes','rx_pkts','tx_pkts','rx_bytes','tx_bytes')

df1.write.parquet('s3://mist-data-science-dev/jing/switch_flat_20190910a')

In [ ]:
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rxpkts", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|545160|
|   2.0| 13594|
|   3.0|   913|
|   4.0| 37937|
|   5.0|  1878|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="txpkts", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|537890|
|   2.0|  2938|
|   3.0|  7942|
|   4.0| 48980|
|   5.0|  1732|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rxbytes", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|542265|
|   3.0|  1445|
|   4.0|  6785|
|   5.0| 48987|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="txbytes", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|533451|
|   4.0|  6592|
|   5.0| 59439|
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="rx_bps", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+ 
|bucket| count|
+------+------+
|   1.0|572735|0-10
|   2.0|   810|
|   3.0|  5103|
|   4.0|  8512|
|   5.0|  8599|
|   6.0|  3225|
|   7.0|   447|
|   8.0|    51|10-100M
+------+------+
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="tx_bps", outputCol="bucket")
rxpkts = bucketizer.transform(df1)
rxpkts.sort('bucket').groupby('bucket').count().show()
+------+------+                                                                 
|bucket| count|
+------+------+
|   1.0|570109|0-10
|   2.0|    85|
|   3.0|  3709|
|   4.0|  4836|
|   5.0| 15945|
|   6.0|  4313|
|   7.0|   435|
|   8.0|    50|10-100M
+------+------+



In [ ]:
df1.groupby(col('id'),col('ts')).agg(sum_('rx_bps').alias('sum_rxbps'),sum_('tx_bps').alias('sum_txbp'),avg('mbps')).show()

max of sum_rxbps: 88M
max of sum_txbps: 91M

In [ ]:
d = dfin.groupby('id','when').agg(sum_('rx_bps'),sum_('tx_bps')) 
splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="sum(rx_bps)", outputCol="bucket")
rxpkts = bucketizer.transform(d)
rxpkts.sort('bucket').groupby('bucket').count().show()

+------+-----+                                                                  
|bucket|count|
+------+-----+
|   1.0| 4303| 
|   4.0|  278|
|   5.0| 5629|
|   6.0| 2382|
|   7.0|  472|
|   8.0|   60| 10-100M
+------+-----+

splits = [-float("inf"), 0.0, 10,100,1000,10000,100000,1000000,10000000,100000000,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="sum(tx_bps)", outputCol="bucket")
rxpkts = bucketizer.transform(d)
rxpkts.sort('bucket').groupby('bucket').count().show()

+------+-----+                                                                  
|bucket|count|
+------+-----+
|   1.0| 4303|
|   4.0|  145|
|   5.0| 4555|
|   6.0| 3528|
|   7.0|  535|
|   8.0|   58| 10-100M
+------+-----+